## import Libaris

In [3]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder , StandardScaler
from pycaret.regression import *

In [ ]:
data = pd.read_csv(r"/kaggle/input/student-performance-factors/StudentPerformanceFactors.csv")

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6607 entries, 0 to 6606
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Hours_Studied               6607 non-null   int64 
 1   Attendance                  6607 non-null   int64 
 2   Parental_Involvement        6607 non-null   object
 3   Access_to_Resources         6607 non-null   object
 4   Extracurricular_Activities  6607 non-null   object
 5   Sleep_Hours                 6607 non-null   int64 
 6   Previous_Scores             6607 non-null   int64 
 7   Motivation_Level            6607 non-null   object
 8   Internet_Access             6607 non-null   object
 9   Tutoring_Sessions           6607 non-null   int64 
 10  Family_Income               6607 non-null   object
 11  Teacher_Quality             6529 non-null   object
 12  School_Type                 6607 non-null   object
 13  Peer_Influence              6607 non-null   obje

In [6]:
data.isnull().sum()

Hours_Studied                  0
Attendance                     0
Parental_Involvement           0
Access_to_Resources            0
Extracurricular_Activities     0
Sleep_Hours                    0
Previous_Scores                0
Motivation_Level               0
Internet_Access                0
Tutoring_Sessions              0
Family_Income                  0
Teacher_Quality               78
School_Type                    0
Peer_Influence                 0
Physical_Activity              0
Learning_Disabilities          0
Parental_Education_Level      90
Distance_from_Home            67
Gender                         0
Exam_Score                     0
dtype: int64

In [7]:
data.dropna(inplace=True)

In [8]:
data.isnull().sum()

Hours_Studied                 0
Attendance                    0
Parental_Involvement          0
Access_to_Resources           0
Extracurricular_Activities    0
Sleep_Hours                   0
Previous_Scores               0
Motivation_Level              0
Internet_Access               0
Tutoring_Sessions             0
Family_Income                 0
Teacher_Quality               0
School_Type                   0
Peer_Influence                0
Physical_Activity             0
Learning_Disabilities         0
Parental_Education_Level      0
Distance_from_Home            0
Gender                        0
Exam_Score                    0
dtype: int64

In [9]:
df_object = data.select_dtypes(include=['object'])
lb = LabelEncoder()
for col in df_object.columns:
    data[col] = lb.fit_transform(data[col])

In [10]:
numeric_cols = data.select_dtypes(include=['int64', 'float64']).columns
z_scores = np.abs((data[numeric_cols] - data[numeric_cols].mean()) / data[numeric_cols].std())

threshold = 3
filtered_data = data[(z_scores < threshold).all(axis=1)]

In [11]:
X = data.drop('Exam_Score' , axis = 1)
y = data['Exam_Score']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
exp = setup(data=X_train, 
      target=y_train,
      normalize=True,
      transformation=True,
      remove_outliers=True,
      imputation_type='simple',  # تعويض القيم المفقودة بالمتوسط/المنوال
      numeric_imputation='mean',
      categorical_imputation='mode'
)

best_model = compare_models()
evaluate_model(best_model)


,Description,Value
0,Session id,8867
1,Target,Exam_Score
2,Target type,Regression
3,Original data shape,"(5102, 20)"
4,Transformed data shape,"(4923, 20)"
5,Transformed train set shape,"(3392, 20)"
6,Transformed test set shape,"(1531, 20)"
7,Numeric features,19
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,0.8591,4.4655,2.0541,0.7130,0.0269,0.0123,0.1230
lightgbm,Light Gradient Boosting Machine,0.8970,4.5299,2.0737,0.7082,0.0273,0.0128,0.6450
huber,Huber Regressor,1.0774,5.0605,2.1983,0.6717,0.0292,0.0155,0.0650
lr,Linear Regression,1.0825,5.0771,2.2030,0.6704,0.0293,0.0156,0.2920
ridge,Ridge Regression,1.0825,5.0770,2.2029,0.6704,0.0293,0.0156,0.0270
lar,Least Angle Regression,1.0825,5.0771,2.2030,0.6704,0.0293,0.0156,0.0310
br,Bayesian Ridge,1.0825,5.0768,2.2029,0.6704,0.0293,0.0156,0.0630
et,Extra Trees Regressor,1.1519,5.2730,2.2564,0.6563,0.0302,0.0167,0.3130
xgboost,Extreme Gradient Boosting,1.0337,5.3262,2.2650,0.6526,0.0301,0.0149,0.1560
rf,Random Forest Regressor,1.2183,5.4912,2.3077,0.6407,0.0310,0.0177,0.3280


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [14]:
predictions = predict_model(best_model , X_test)

In [15]:
predictions.prediction_label[120:130]

5351    64.201131
5052    70.888328
1810    67.230761
1759    67.306938
1034    70.678136
4463    69.510039
266     69.112448
3892    62.458336
4097    63.076465
5580    68.138914
Name: prediction_label, dtype: float64

In [16]:
y_test[120:130]

5351    64
5052    71
1810    67
1759    66
1034    71
4463    70
266     69
3892    61
4097    63
5580    69
Name: Exam_Score, dtype: int64

In [17]:
fig = px.scatter(
    x = predictions.prediction_label,
    y = y_test,
    labels={'x':'Prediction' ,'y': 'True'}
)

fig.show()

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [19]:
classifiers = [
    LinearRegression(),
    RandomForestRegressor(),
    XGBRegressor(),
    SVR(),
    KNeighborsRegressor(),
    Lasso(),
    GradientBoostingRegressor(learning_rate= 0.1, max_depth= 3, n_estimators= 200)
]

results = {}
R2 = {}
for model in classifiers:
    model_name = model.__class__.__name__
    print(f"\nTraining {model_name}...")

    cv_scores = cross_val_score(model, X_train, y_train, cv=10, scoring='r2')
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    results[model_name] = {
        "CV R²": cv_scores.mean(),
        "Test R²": r2_score(y_test, y_pred),
        "MAE": mean_absolute_error(y_test, y_pred),
        "MSE": mean_squared_error(y_test, y_pred)
    }
    R2[model_name] = r2_score(y_test,y_pred)


Training LinearRegression...

Training RandomForestRegressor...

Training XGBRegressor...

Training SVR...

Training KNeighborsRegressor...

Training Lasso...

Training GradientBoostingRegressor...


In [20]:
results_df = pd.DataFrame(results).T  
print(results_df.to_string(float_format="%.4f")) 

                           CV R²  Test R²    MAE    MSE
LinearRegression          0.6467   0.6644 1.0642 5.2152
RandomForestRegressor     0.6211   0.6190 1.1841 5.9211
XGBRegressor              0.6302   0.6386 1.0037 5.6161
SVR                       0.6017   0.6178 1.2715 5.9393
KNeighborsRegressor       0.5106   0.5222 1.5453 7.4247
Lasso                     0.5671   0.5800 1.4358 6.5268
GradientBoostingRegressor 0.6972   0.7097 0.7156 4.5111


In [21]:
best_model = max(R2, key=R2.get)
best_model , R2[best_model]

('GradientBoostingRegressor', 0.7096951857560011)

In [22]:
from sklearn.model_selection import GridSearchCV

# تعريف معاملات الضبط
params = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

# البحث عن أفضل معاملات
gb = GradientBoostingRegressor()
grid_search = GridSearchCV(gb, params, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)

print("أفضل معلمات:", grid_search.best_params_)

أفضل معلمات: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}


In [23]:
model = GradientBoostingRegressor(learning_rate= 0.1, max_depth= 3, n_estimators= 200)
model.fit(X_train , y_train)

y_pred = model.predict(X_test)

In [24]:
r2_score(y_test , y_pred)

0.7097787630786073

In [25]:
y_pred[120:130]

array([63.67928056, 73.74582321, 67.99394833, 66.65465414, 70.99237815,
       70.36380903, 69.09362509, 61.95053065, 63.31496185, 69.11262408])

In [26]:
y_test[120:130]

5351    64
5052    71
1810    67
1759    66
1034    71
4463    70
266     69
3892    61
4097    63
5580    69
Name: Exam_Score, dtype: int64

In [27]:
fig = px.scatter(x = y_test , 
                 y = y_pred,
                 labels={'x':'True' , 'y' : 'prediction'} )
fig.show()